In [1]:
import json
import sqlalchemy
from sqlalchemy import UniqueConstraint, CheckConstraint
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
from sqlalchemy import Column, Integer, String, Text, Enum, Float, Boolean
from sqlalchemy import ForeignKey, ForeignKeyConstraint
from sqlalchemy.orm import relationship

from sqlalchemy import event
from sqlalchemy.engine import Engine
from sqlite3 import Connection as SQLite3Connection

@event.listens_for(Engine, "connect")
def _set_sqlite_pragma(dbapi_connection, connection_record):
    if isinstance(dbapi_connection, SQLite3Connection):
        cursor = dbapi_connection.cursor()
        cursor.execute("PRAGMA foreign_keys=ON;")
        cursor.close()
        
from sqlalchemy import create_engine
engine = create_engine('sqlite:///params.sqlite3', echo=False)

In [3]:
class Unit(Base):
    __tablename__ = 'ex_units'
    
    id = Column(Integer, primary_key=True)
    whp_unit = Column(String, nullable=True, unique=True)
    cf_unit = Column(String, nullable=False)
    reference_scale = Column(String, nullable=True)
    note = Column(Text, nullable=True)
    
class Param(Base):
    __tablename__ = 'ex_params'
    whp_name = Column(String, primary_key=True)
    whp_number = Column(Integer, nullable=True)
    description = Column(Text, nullable=True)
    note = Column(Text, nullable=True)
    warning = Column(Text, nullable=True)
    
    scope = Column(Enum('cruise', 'profile', 'sample'), nullable=False, server_default='sample')
    dtype = Column(Enum('decimal', 'integer', 'string'), nullable=False, )
    flag = Column(Enum('woce_bottle', 'woce_ctd', 'woce_discrete', 'no_flags'), nullable=False)
    ancillary = Column(Boolean, nullable=False, server_default='0')
    
    rank = Column(Float, nullable=False)
    

class CFName(Base):
    __tablename__ = 'cf_names'
    
    standard_name = Column(String, primary_key=True)
    canonical_units = Column(String, nullable=True)
    grib = Column(String, nullable=True)
    amip = Column(String, nullable=True)
    description = Column(Text, nullable=True)
    
class CFAlias(Base):
    __tablename__ = 'cf_aliases'
    
    id = Column(Integer, primary_key=True) # cannot use numeric id since alias isn't unique
    alias = Column(String, nullable=False)
    standard_name = Column(String, ForeignKey(CFName.__table__.c.standard_name), nullable=False)
    
class WHPName(Base):
    __tablename__ = 'whp_names'
    
    whp_name = Column(String, ForeignKey(Param.__table__.c.whp_name), primary_key=True)
    whp_unit = Column(String, ForeignKey(Unit.__table__.c.whp_unit), primary_key=True, nullable=True)
    standard_name = Column(String, ForeignKey(CFName.__table__.c.standard_name), nullable=True)
    nc_name = Column(String, unique=True, nullable=True)
    
    numeric_min = Column(Float, nullable=True)
    numeric_max = Column(Float, nullable=True)
    
    error_name = Column(String, nullable=True)
    
    analytical_temperature_name = Column(String, nullable=True)
    analytical_temperature_units = Column(String, nullable=True)
    
    field_width = Column(Integer, nullable=False)
    numeric_precision = Column(Integer, nullable=True)
    
    __table_args__ = (
        ForeignKeyConstraint(
            ['analytical_temperature_name', 'analytical_temperature_units'],
            ['whp_names.whp_name', 'whp_names.whp_unit'],
        ),
    )
    
class Alias(Base):
    __tablename__ = "whp_alias"
    old_name = Column(String, primary_key=True)
    old_unit = Column(String, primary_key=True, nullable=True)
    whp_name = Column(String)
    whp_unit = Column(String)
    
    __table_args__ = (
        ForeignKeyConstraint(
            ['whp_name', 'whp_unit'],
            ['whp_names.whp_name', 'whp_names.whp_unit'],
        ),
    )
    

In [4]:
Base.metadata.create_all(engine)

In [5]:
from sqlalchemy.orm import sessionmaker
from xml.etree import ElementTree
Session = sessionmaker(bind=engine)
session = Session()


In [6]:
cf_names = []
cf_aliases = []
version_number = None
for element in ElementTree.parse("cf-standard-name-table.xml").getroot():
    if element.tag == "version_number":
        version_number = int(element.text)
    if element.tag not in ("entry", "alias"):
        continue

    name = element.attrib["id"]
    name_info = {info.tag: info.text for info in element}

    if element.tag == "entry":
        cf_names.append(CFName(standard_name=name, **name_info))

    if element.tag == "alias":
        cf_aliases.append(CFAlias(alias=name, standard_name=name_info["entry_id"]))

In [7]:
with open("parameters.json") as f:
    params = json.load(f)
units = {p["whp_unit"]:p.get("cf_unit") for p in params}
refscales = {p["whp_unit"]:p.get("reference_scale") for p in params}

unit_list = []
for key, value in units.items():
    cf_unit = value
    if key is None:
        unit_list.append(Unit(whp_unit=key, cf_unit="1"))
        continue
    if cf_unit is None:
        cf_unit = key.lower()
    unit_list.append(Unit(whp_unit=key, cf_unit=cf_unit, reference_scale=refscales[key]))
    

session.add_all(cf_names)
session.commit()
session.add_all(cf_aliases)
session.commit()
session.add_all(unit_list)
session.commit()

In [8]:
whp_name = []
db_params = []
rank = 1
for param in params:
    if param['whp_name'] in whp_name:
        continue
    whp_name.append(param['whp_name'])
    
    flag = param["flag_w"]
    if flag == None:
        flag = "no_flags"
    db_params.append(Param(whp_name = param['whp_name'],
    whp_number=(param.get("whp_number")),
    description=(param.get("description")),
    note=(param.get("note")),
    warning=(param.get("warning")),
    
    scope=(param.get("scope", "sample")),
    dtype=(param["data_type"]),
    flag=flag,
    rank=rank,
    ancillary=False,
         ))
    rank+=1
session.add_all(db_params)
session.commit()

In [9]:
whp_params = []
for param in params:
    whp_params.append(WHPName(
        whp_name=param['whp_name'],
        whp_unit=param['whp_unit'],
        standard_name=param.get('cf_name'),
        nc_name=None, #this is a todo
        numeric_min=param.get('numeric_min'),
        numeric_max=param.get('numeric_max'),
        field_width=param.get('field_width'),
        numeric_precision=param.get('numeric_precision'),
        error_name=param.get("error_name")
    ))

In [10]:
session.add_all(whp_params)
session.commit()

In [11]:
with open("aliases.json") as f:
    aliases = json.load(f)

In [12]:
alias_ad = []
for alias in aliases:
    alias_ad.append(
        Alias(
            old_name = alias["whp_name"],
            old_unit = alias["whp_unit"],
            whp_name = alias["canonical_name"],
            whp_unit = alias["canonical_unit"],
        )
    )

In [13]:
session.add_all(alias_ad)

In [14]:
session.commit()